In [1]:
import pandas as pd

file_path="csv/GroundTruthV2.csv"

df = pd.read_csv(file_path, sep=";")

# Extract columns into separate arrays
art_ids = df.iloc[:, 0].tolist()  # First column
topics = df.iloc[:, 1].tolist()  # Second column

art_ids = [str(number).zfill(8) for number in art_ids] #Add 0 padding for articleIDs

# Print the arrays to verify
print("Art Ids:", art_ids)

Art Ids: ['70301542', '19294535', '89442655', '90301555', '69329387', '79442689', '39442672', '09442678', '90510865', '89278259', '29278304', '80401292', '29442615', '69442680', '50454507', '19189030', '59442685', '49442676', '70538846', '19442654', '29481701', '40501892', '49189203', '80487813', '19442692', '59189472', '79278250', '59294557', '99017445', '79442694']


In [2]:
from google.cloud import bigquery
client = bigquery.Client()

all_reviews = []

for art_id in art_ids:
    query = f"""SELECT concat(title,'. ',text) as text FROM `ingka-feed-student-dev.RR.RatingsReviews` AS rr
                INNER JOIN `ingka-feed-student-dev.RR.product_categories` AS pc 
                ON rr.art_id = SPLIT(pc.global_id, ',')[SAFE_OFFSET(1)] 
                WHERE country_code = 'us' and PRODUCT_AREA = 'Open storage' and art_id = '{art_id}'
                ORDER BY inserted_on DESC """

    query_job = client.query(query)

    article_reviews = []

    for review in query_job:
         article_reviews.append(review.text)

    all_reviews.append(article_reviews)
len(all_reviews)

30

In [3]:
import pandas as pd

file_path="csv/0.6TopP_5ShotLlama3+Keyphrases_Topics.csv"

df = pd.read_csv(file_path, sep=";")

# Extract columns into separate arrays
all_topics = df.iloc[:, 1].tolist() 

In [4]:
"""from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch

# Load a sentence transformer model
model = SentenceTransformer('Embedding-model/mpnetv2')

def find_reviews_with_topic(reviews, topic_phrase, max_reviews=4):
    topic_reviews = []

    # Embed the topic phrase
    topic_embedding = model.encode(topic_phrase, convert_to_tensor=True)

    # Iterate through each review
    for review in reviews:
        # Break the review into sentences and encode
        sentences = review.split('. ')
        sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(topic_embedding, sentence_embeddings)

        # Check if any sentence has a similarity above the threshold
        max_similarity, max_index = torch.max(similarities, dim=1)  # Find max similarity and its index
        if max_similarity > 0.8:  # Using a threshold for similarity
            highlighted_sentence = sentences[max_index.item()].replace(topic_phrase, f"**{topic_phrase}**")
            topic_reviews.append(review.replace(sentences[max_index.item()], f"**{highlighted_sentence}**"))
        
        if len(topic_reviews) >= 4:
            break

    return topic_reviews

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['articleID', 'topic', 'reviews'])
        for item in data:
            writer.writerow(item)

csv_data = []
i=0
for reviews, art_id in zip(all_reviews, art_ids):
    topics = all_topics[i].split(', ')
    i+=1
    for topic in topics:
        matched_reviews = find_reviews_with_topic(reviews, topic)
        csv_data.append([art_id, topic, '| '.join(matched_reviews)]) """

'from sentence_transformers import SentenceTransformer, util\nimport numpy as np\nimport torch\n\n# Load a sentence transformer model\nmodel = SentenceTransformer(\'Embedding-model/mpnetv2\')\n\ndef find_reviews_with_topic(reviews, topic_phrase, max_reviews=4):\n    topic_reviews = []\n\n    # Embed the topic phrase\n    topic_embedding = model.encode(topic_phrase, convert_to_tensor=True)\n\n    # Iterate through each review\n    for review in reviews:\n        # Break the review into sentences and encode\n        sentences = review.split(\'. \')\n        sentence_embeddings = model.encode(sentences, convert_to_tensor=True)\n        similarities = util.pytorch_cos_sim(topic_embedding, sentence_embeddings)\n\n        # Check if any sentence has a similarity above the threshold\n        max_similarity, max_index = torch.max(similarities, dim=1)  # Find max similarity and its index\n        if max_similarity > 0.8:  # Using a threshold for similarity\n            highlighted_sentence = se

In [5]:
"""import csv

# Save to CSV
save_to_csv(csv_data, 'csv/topic_reviews08.csv')
print("Data has been saved to 'topic_reviews08.csv'.") """    

'import csv\n\n# Save to CSV\nsave_to_csv(csv_data, \'csv/topic_reviews08.csv\')\nprint("Data has been saved to \'topic_reviews08.csv\'.") '

In [6]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
import csv

# Load a sentence transformer model
model = SentenceTransformer('Embedding-model/mpnetv2')

# Check if a GPU is available and use it
device ='cpu'
model = model.to(device)


def find_reviews_with_topic(reviews, topic_phrase, max_reviews=4):
    topic_reviews = []

    # Embed the topic phrase
    topic_embedding = model.encode(topic_phrase, convert_to_tensor=True).to(device)
    # Iterate through each review
    for review in reviews:
        # Break the review into sentences
        sentences = review.split('. ')
        highlighted_review = review
        found = False

        for sentence in sentences:
            
            words = sentence.split()
            
            if not words:
                continue

            word_embeddings = model.encode(words, convert_to_tensor=True).to(device)
            if word_embeddings.shape[0] == 0:
                continue

            similarities = util.pytorch_cos_sim(topic_embedding, word_embeddings)
            max_similarity, max_index = torch.max(similarities, dim=1)
            if max_similarity > 0.80:
                index = max_index.item()
                highlighted_word = words[index]
                start_index = index
                end_index = index

                # Check if the highlighted word contains any special characters
                contains_special_char = any(char in highlighted_word for char in [',', ';', '.'])

                # Expand to the left
                left_count = 0
                while start_index > 0 and not any(char in words[start_index - 1] for char in [',', ';', '.', '!', '?']) and left_count < 3:
                    start_index -= 1
                    left_count += 1

                # Expand to the right only if no special characters in the highlighted word
                if not contains_special_char:
                    right_count = 0
                    while end_index < len(words) - 1 and words[end_index + 1] not in [',', ';', '.','!','?'] and right_count < 3:
                        end_index += 1
                        right_count += 1

                # Form the highlighted phrase and update the sentence
                highlighted_phrase = ' '.join(words[start_index:end_index + 1])
                highlighted_sentence = sentence.replace(highlighted_phrase, f"**{highlighted_phrase}**")
                highlighted_review = highlighted_review.replace(sentence, highlighted_sentence)
                found = True


        if found:
            topic_reviews.append(highlighted_review)

        if len(topic_reviews) >= max_reviews:
            break

    return topic_reviews


def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['ArticleID', 'Topic', 'Reviews'])
        for item in data:
            writer.writerow(item)

csv_data = []
i = 0
for reviews, art_id in zip(all_reviews, art_ids):
    print(art_id)
    topics = all_topics[i].split(', ')
    i += 1
    for topic in topics:
        matched_reviews = find_reviews_with_topic(reviews, topic)
        csv_data.append([art_id, topic, '| '.join(matched_reviews)])

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


70301542
19294535
89442655
90301555
69329387
79442689
39442672
09442678
90510865
89278259
29278304
80401292
29442615
69442680
50454507
19189030
59442685
49442676
70538846
19442654
29481701
40501892
49189203
80487813
19442692
59189472
79278250
59294557
99017445
79442694


In [7]:
import csv

filename = 'csv/final_topic_reviews_word080.csv'

# Save to CSV
save_to_csv(csv_data, filename)
print(f"Data has been saved to {filename}.")

Data has been saved to csv/final_topic_reviews_word080.csv.
